In [ ]:
import argparse
import numpy as np
import pandas as pd
from multiprocessing import cpu_count
from sklearn.model_selection import train_test_split
import re
import os
import datetime

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
%load_ext tensorboard

In [ ]:
window = 5 # depends on time window
epochs = 100
batch_size = 16
pose_vec_dim = 36 # depends on pose estimation model used

In [ ]:
class_names = [0, 1, 2]
num_class = len(class_names)
lbl_dict = {class_name:idx for idx, class_name in enumerate(class_names)}

In [ ]:
csv_path = './data/csv/result.csv'
validate_csv_path = './data/csv/validate.csv'

In [ ]:
dataset = pd.read_csv(csv_path,  index_col=None)

In [ ]:
def convert_vec(x):
    return np.array(re.split(',*',re.sub('^\D\D*|\D*\D$', '',re.sub('\D', ',', str(x)))), dtype=float)

In [ ]:
def shape_data(X, Y, window):
    new_x = []
    new_y = []
    max_row = X.shape[0] - (X.shape[0] % window + 1)
    i = window
    while i < max_row:
        new_x.append(X[i-window+1:i+1]-X[i-window:i])
        new_y.append(Y[i])
        i+=1
    return np.array(new_x), np.array(new_y)

In [ ]:
dataset = pd.read_csv(csv_path,  index_col=None)

y = dataset.label.values
X = np.stack(dataset.vec.apply(convert_vec).values)

In [ ]:
min_max_scaler = MinMaxScaler().fit(X)

X = min_max_scaler.transform(X)

X,y = shape_data(X,y,window)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

y_train = tf.keras.utils.to_categorical(list(map(lbl_dict.get, y_train)), num_class)
y_test = tf.keras.utils.to_categorical(list(map(lbl_dict.get, y_test)), num_class)

#y_valid = y_test[:y_test.shape[0] // 3]
#X_valid = X_test[:X_test.shape[0] // 3]

#y_test = y_test[y_test.shape[0] // 3:]
#X_test = X_test[X_test.shape[0] // 3:]

#X_test = X_test.reshape(X_test.shape[0], pose_vec_dim, window)
#X_train = X_train.reshape(X_train.shape[0], pose_vec_dim, window)

In [ ]:
import pickle
with open('min_max_scaler.pickle', 'wb') as f:
    pickle.dump(min_max_scaler, f)

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
checkpointer = ModelCheckpoint(filepath="../app/models/lstm_model.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

In [ ]:
model = Sequential()
model.add(LSTM(32, recurrent_dropout=0.5, input_shape=(window, pose_vec_dim)))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(class_names), activation='softmax'))
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

In [ ]:
%tensorboard --logdir logs --port 6006 --bind_all

In [ ]:
history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(X_test, y_test),
                        callbacks=[checkpointer, tensorboard_callback])

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Ошибка')
plt.ylabel('Ошибка')
plt.xlabel('Эпоха')
plt.legend(['Обучающая выборка', 'Тестовая выборка'], loc='best')
plt.show()

plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Точность')
plt.ylabel('Точность')
plt.xlabel('Эпоха')
plt.legend(['Обучающая выборка', 'Тестовая выборка'], loc='best')
plt.show()

In [ ]:
dataset = pd.read_csv(validate_csv_path,  index_col=None)

y_valid = dataset.label.values
X_valid = np.stack(dataset.vec.apply(convert_vec).values)

X_valid = min_max_scaler.transform(X_valid)

In [ ]:
X_valid,y_valid = shape_data(X_valid,y_valid,window)

y_valid = tf.keras.utils.to_categorical(list(map(lbl_dict.get, y_valid)), num_class)

In [ ]:
model_lstm = tf.keras.models.load_model('../app/models/lstm_model.h5')
model_gru = tf.keras.models.load_model('../app/models/gru_model.h5')

In [ ]:
%%time
model_lstm.evaluate(X_valid, y_valid)

In [ ]:
%%time
model_gru.evaluate(X_valid, y_valid)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
lstm_predict = np.argmax(model_lstm.predict(X_valid), 1)
gru_predict = np.argmax(model_gru.predict(X_valid), 1)

In [ ]:
print("Gru f1-score: {0}".format(f1_score(np.argmax(y_valid, 1), lstm_predict, average='weighted')))
print("LSTM f1-score: {0}".format(f1_score(np.argmax(y_valid, 1), gru_predict, average='weighted')))